In [1]:
import io
def get_unigrams(file_name):
    unigrams = {}
    count = 0
    with io.open(file_name, encoding='utf8', errors='ignore') as f:
        for line in f:
            tokens = line.strip().split()
            count+=1
            for token in tokens:
                token = token.lower()
                try:
                    unigrams[token]
                except:
                    unigrams[token] = 0
                unigrams[token] += 1
                
    return unigrams

def index_unigrams(unigrams):
    new_unigrams = {}
    reverse_unigrams = {}
    for index, unigram in enumerate(unigrams):
        new_unigrams[unigram] = index
        reverse_unigrams[index] = unigram
    return new_unigrams, reverse_unigrams
            

In [28]:
file_name = "NNN/sample_corpus/sample_corpus.txt"
import nltk
import copy
unigrams = get_unigrams(file_name)
words = [i for i in unigrams.keys()]
pos = nltk.pos_tag(words)
verbs = [i[0] for i in pos if i[1]=='VB' or i[1]=='VBD' or i[1]=='VBG' or i[1]=='VBN' or i[1]=='VBP' or i[1]=='VBZ']
iunigrams,runigrams = index_unigrams(unigrams)
unigrams = sorted(unigrams.items(), key = lambda x: x[1], reverse = True )
print (unigrams[0])
#unigrams = [i for i in unigrams if i[0] in verbs]
from pprint import pprint
#pprint.pprint(iunigrams) # Figure out non-stop words
# dimensions = [x[0] for x in unigrams[100:1600]]
count = 0
dimensions = list()
for x in unigrams[100:]:
    if x[0] in verbs:
        dimensions.append(x[0])
        count += 1
    if count == 3000:
        break
idimensions = {x: index for index, x in enumerate(dimensions)}
#pprint(dimensions)
# print(dimensions.shape)



('the', 26256)


In [29]:
import numpy
cmatrix = numpy.memmap("lsa.cmatrix", dtype='float64', mode='w+', shape=(len(unigrams),len(dimensions)))
print(cmatrix.shape)

(43124, 6000)


In [35]:
def populate_cmatrix(file_name, cmatrix, iunigrams, dimensions, window = 5):
     e = 0
     s = 0
     with open(file_name, encoding='utf-8', errors='ignore') as f:

        count = 0
        for index, line in enumerate(f):
            tokens = line.strip().split()
            posTokens = nltk.pos_tag(tokens)
            postokens = [i[0] for i in posTokens if i[1]=='VB' or i[1]=='VBD' or i[1]=='VBG' or i[1]=='VBN' or i[1]=='VBP' or i[1]=='VBZ']
            count+=1
            for indexj, token in enumerate(tokens):
                token = token.lower()
                lcontext = tokens[indexj - window:indexj]
                rcontext = tokens[indexj + 1:index + window]
                context = [tok.lower() for tok in lcontext + rcontext]
                
                try:
                    unigram_index = iunigrams[token]                    
                    for d in context:
                        
                        if d in dimensions:
                            j = dimensions[d]
                            cmatrix[unigram_index][j] += 1
#                     for d in context:
                        
#                         if d in dimensions:
#                             j = dimensions[d]
#                             cmatrix[unigram_index][1500+j] += 1
                            s += 1                          
                except:
                    e += 1
            
            
     print(e,s)
                
                

In [36]:
from time import time
s = time()
populate_cmatrix(file_name, cmatrix, iunigrams, idimensions)
e = time()
print(e -s)

0 769283
40.21617412567139


In [37]:
#boy, sunday, eat, good, slowly, 100 
w1 = 'boy'
w2 = 'sunday'
w3 = 'eat'
w4 = 'good'
w5 = 'slowly'
w6 = '100'
id1 = iunigrams[w1]
id2 = iunigrams[w2]
id3 = iunigrams[w3]
id4 = iunigrams[w4]
id5 = iunigrams[w5]
id6 = iunigrams[w6]
print(id1, id2, id3, id4, id5)
v1 = cmatrix[id1]
v2 = cmatrix[id2]
v3 = cmatrix[id3]

print(v1, v2, v3)
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import *
print(euclidean(v1, v2))
print(cosine(v1,v2))
a = ((v1.dot(v1))/(numpy.linalg.norm(v1)*numpy.linalg.norm(v1)))
print (1-a)

12636 5130 191 816 8607
[ 0.  0.  0. ...,  0.  0.  0.] [ 0.  0.  3. ...,  0.  0.  0.] [ 0.  0.  0. ...,  0.  0.  0.]
41.677331968349414
0.920444271582
0.0


In [38]:
from sklearn.decomposition import TruncatedSVD
s = time()
svd = TruncatedSVD(n_components=100, random_state=42)
svd.fit(cmatrix)
twod_cmatrix = svd.transform(cmatrix)
e = time()
print(e - s )

47.25409913063049


In [39]:
v1_2d, v2_2d, v3_2d, v4_2d, v5_2d, v6_2d = twod_cmatrix[id1], twod_cmatrix[id2], twod_cmatrix[id3], twod_cmatrix[id4], twod_cmatrix[id5], twod_cmatrix[id6]
print ([runigrams[j] for i,j in enumerate([id1,id2,id3,id4,id5,id6])])
def getTen(vec):
    cosi = []
    for i in range(len(twod_cmatrix)):
        if numpy.linalg.norm(twod_cmatrix[i]) == 0:
            continue
        cosi.append((i,cosine(twod_cmatrix[i],vec)))
    cosi = sorted(cosi,key=lambda x: x[1])
    print (runigrams[cosi[0][0]],cosi[0][1], cosine(v1_2d,v1_2d))
    return [i[0] for i in cosi[1:11]]
for i in [v1_2d, v2_2d, v3_2d, v4_2d, v5_2d, v6_2d]:
    indx = getTen(i)
    print ([runigrams[i] for i in indx])


['boy', 'sunday', 'eat', 'good', 'slowly', '100']
boy 1.11022302463e-16 1.11022302463e-16
['ou', 'mens', 'widow', 'son', 'bullies', 'janet', 'carrier', 'flat', 'dogged', 'lap']
sunday 1.11022302463e-16 1.11022302463e-16
['last', 'spoke', 'week', 'the', 'his', 'at', 'elections', 'on', 'deal', ',']
eat 0.0 1.11022302463e-16
['spilled', 'peterborough', 'humorous', '2.2.4ac1', 'hoylake', 'joking', 'videotape', 'eventing', 'breisner', 'handball']
good 0.0 1.11022302463e-16
['it', 'way', 'but', 'find', 'say', 'like', 'then', 'now', 'get', 'so']
slowly -2.22044604925e-16 1.11022302463e-16
['executives', 'emissions', 'fuels', 'dioxide', 'balanced', 'commanders', 'controversies', 'c.i.a.', 'pentagon', 'iran']
100 -4.4408920985e-16 1.11022302463e-16
['=', '10', 'centimetres', 'sq.', 'metric', 'millimetres', 'centimetre', 'decimeter', 'decimetres', 'decametres']


In [ ]:
%pylab inline
import matplotlib.pyplot as plt
v1_2d = v1_2d / numpy.linalg.norm(v1_2d)
v2_2d = v2_2d / numpy.linalg.norm(v2_2d)
v3_2d = v3_2d / numpy.linalg.norm(v3_2d)
print ([v1_2d, v2_2d,v3_2d])
colors = ['r','b','g']
fig, axs = plt.subplots(1,1)
for i, x in enumerate([v1_2d, v2_2d,v3_2d]):
    a = plt.plot([0,x[0]],[0,x[1]],colors[i]+'-')
plt.show()

In [10]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20,n_init=100,verbose=True)
temp = kmeans.fit(twod_cmatrix)
print (twod_cmatrix.shape)

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 16344298.8864
start iteration
done sorting
end inner loop
Iteration 1, inertia 16013970.7418
start iteration
done sorting
end inner loop
Iteration 2, inertia 15945163.234
start iteration
done sorting
end inner loop
Iteration 3, inertia 15934675.1423
start iteration
done sorting
end inner loop
Iteration 4, inertia 15871089.6068
start iteration
done sorting
end inner loop
Iteration 5, inertia 15849166.2273
start iteration
done sorting
end inner loop
Iteration 6, inertia 15849166.2273
center shift 0.000000e+00 within tolerance 3.939571e-03
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 16544635.8662
start iteration
done sorting
end inner loop
Iteration 1, inertia 16209136.94
start iteration
done sorting
end inner loop
Iteration 2, inertia 16163658.8235
start iteration
done sorting
end inner loop
Iteration 3, inertia 16144337.5018
start iteration
done sorting
e

done sorting
end inner loop
Iteration 0, inertia 17740933.8634
start iteration
done sorting
end inner loop
Iteration 1, inertia 17389486.3516
start iteration
done sorting
end inner loop
Iteration 2, inertia 17119488.6991
start iteration
done sorting
end inner loop
Iteration 3, inertia 16856064.7458
start iteration
done sorting
end inner loop
Iteration 4, inertia 16708828.7463
start iteration
done sorting
end inner loop
Iteration 5, inertia 16687762.1308
start iteration
done sorting
end inner loop
Iteration 6, inertia 16677017.0862
start iteration
done sorting
end inner loop
Iteration 7, inertia 16672717.7759
start iteration
done sorting
end inner loop
Iteration 8, inertia 16670315.5198
start iteration
done sorting
end inner loop
Iteration 9, inertia 16668838.3282
start iteration
done sorting
end inner loop
Iteration 10, inertia 16667133.6765
start iteration
done sorting
end inner loop
Iteration 11, inertia 16666582.6417
start iteration
done sorting
end inner loop
Iteration 12, inertia 

Iteration 5, inertia 15861401.1454
center shift 0.000000e+00 within tolerance 3.939571e-03
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 15560264.3906
start iteration
done sorting
end inner loop
Iteration 1, inertia 15411278.8598
start iteration
done sorting
end inner loop
Iteration 2, inertia 15402845.55
start iteration
done sorting
end inner loop
Iteration 3, inertia 15402845.55
center shift 0.000000e+00 within tolerance 3.939571e-03
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 16107289.2567
start iteration
done sorting
end inner loop
Iteration 1, inertia 15872619.215
start iteration
done sorting
end inner loop
Iteration 2, inertia 15786233.5774
start iteration
done sorting
end inner loop
Iteration 3, inertia 15626298.2729
start iteration
done sorting
end inner loop
Iteration 4, inertia 15568411.2389
start iteration
done sorting
end inner loop
Iteration 5, inertia 15544413.0119
start iteration
done

Iteration 9, inertia 16944664.0107
start iteration
done sorting
end inner loop
Iteration 10, inertia 16944664.0107
center shift 0.000000e+00 within tolerance 3.939571e-03
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 16291774.0272
start iteration
done sorting
end inner loop
Iteration 1, inertia 15640133.7987
start iteration
done sorting
end inner loop
Iteration 2, inertia 15328312.5219
start iteration
done sorting
end inner loop
Iteration 3, inertia 15303280.0329
start iteration
done sorting
end inner loop
Iteration 4, inertia 15302356.2181
start iteration
done sorting
end inner loop
Iteration 5, inertia 15298797.5937
start iteration
done sorting
end inner loop
Iteration 6, inertia 15297702.6255
start iteration
done sorting
end inner loop
Iteration 7, inertia 15297702.6255
center shift 0.000000e+00 within tolerance 3.939571e-03
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 15854998.0812
start iteratio

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 17786930.1843
start iteration
done sorting
end inner loop
Iteration 1, inertia 17419894.0787
start iteration
done sorting
end inner loop
Iteration 2, inertia 17393098.8075
start iteration
done sorting
end inner loop
Iteration 3, inertia 17387911.6086
start iteration
done sorting
end inner loop
Iteration 4, inertia 17386673.5272
start iteration
done sorting
end inner loop
Iteration 5, inertia 17386397.0391
start iteration
done sorting
end inner loop
Iteration 6, inertia 17385588.8122
start iteration
done sorting
end inner loop
Iteration 7, inertia 17384445.0407
start iteration
done sorting
end inner loop
Iteration 8, inertia 17382987.3852
start iteration
done sorting
end inner loop
Iteration 9, inertia 17382614.7992
start iteration
done sorting
end inner loop
Iteration 10, inertia 17381600.5324
start iteration
done sorting
end inner loop
Iteration 11, inertia 17379836.8553
start iteration
done sorti

...

Iteration 13, inertia 15333774.3442
center shift 0.000000e+00 within tolerance 3.939571e-03
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 17837384.0177
start iteration
done sorting
end inner loop
Iteration 1, inertia 17052271.2387
start iteration
done sorting
end inner loop
Iteration 2, inertia 16976885.5834
start iteration
done sorting
end inner loop
Iteration 3, inertia 16896222.3632
start iteration
done sorting
end inner loop
Iteration 4, inertia 16888280.6796
start iteration
done sorting
end inner loop
Iteration 5, inertia 16885462.4126
start iteration
done sorting
end inner loop
Iteration 6, inertia 16884501.8156
start iteration
done sorting
end inner loop
Iteration 7, inertia 16883981.0308
start iteration
done sorting
end inner loop
Iteration 8, inertia 16883710.4977
start iteration
done sorting
end inner loop
Iteration 9, inertia 16883524.5499
start iteration
done sorting
end inner loop
Iteration 10, inertia 16883520.4336
start iteratio

Iteration 13, inertia 15672266.6296
start iteration
done sorting
end inner loop
Iteration 14, inertia 15606622.5958
start iteration
done sorting
end inner loop
Iteration 15, inertia 15604459.1426
start iteration
done sorting
end inner loop
Iteration 16, inertia 15602234.8358
start iteration
done sorting
end inner loop
Iteration 17, inertia 15601573.8773
start iteration
done sorting
end inner loop
Iteration 18, inertia 15601442.4339
start iteration
done sorting
end inner loop
Iteration 19, inertia 15601442.4339
center shift 0.000000e+00 within tolerance 3.939571e-03
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 17382824.0817
start iteration
done sorting
end inner loop
Iteration 1, inertia 16876441.4957
start iteration
done sorting
end inner loop
Iteration 2, inertia 16676239.0467
start iteration
done sorting
end inner loop
Iteration 3, inertia 16500575.4991
start iteration
done sorting
end inner loop
Iteration 4, inertia 16378042.2173
start ite

Iteration 7, inertia 15794576.3866
start iteration
done sorting
end inner loop
Iteration 8, inertia 15794576.3866
center shift 0.000000e+00 within tolerance 3.939571e-03
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 16993038.1611
start iteration
done sorting
end inner loop
Iteration 1, inertia 16513454.819
start iteration
done sorting
end inner loop
Iteration 2, inertia 16296301.9065
start iteration
done sorting
end inner loop
Iteration 3, inertia 16079366.4744
start iteration
done sorting
end inner loop
Iteration 4, inertia 16052083.1636
start iteration
done sorting
end inner loop
Iteration 5, inertia 16047170.9427
start iteration
done sorting
end inner loop
Iteration 6, inertia 16042013.0687
start iteration
done sorting
end inner loop
Iteration 7, inertia 16039388.8986
start iteration
done sorting
end inner loop
Iteration 8, inertia 16037384.8803
start iteration
done sorting
end inner loop
Iteration 9, inertia 16037111.5843
start iteration
d

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 16157450.3705
start iteration
done sorting
end inner loop
Iteration 1, inertia 15858260.8472
start iteration
done sorting
end inner loop
Iteration 2, inertia 15809413.8996
start iteration
done sorting
end inner loop
Iteration 3, inertia 15784751.7264
start iteration
done sorting
end inner loop
Iteration 4, inertia 15721074.2313
start iteration
done sorting
end inner loop
Iteration 5, inertia 15664196.7922
start iteration
done sorting
end inner loop
Iteration 6, inertia 15629821.0829
start iteration
done sorting
end inner loop
Iteration 7, inertia 15628670.57
start iteration
done sorting
end inner loop
Iteration 8, inertia 15628387.3604
start iteration
done sorting
end inner loop
Iteration 9, inertia 15628161.9133
start iteration
done sorting
end inner loop
Iteration 10, inertia 15628068.251
start iteration
done sorting
end inner loop
Iteration 11, inertia 15628014.8107
start iteration
done sorting


Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 16609805.0074
start iteration
done sorting
end inner loop
Iteration 1, inertia 16078706.5976
start iteration
done sorting
end inner loop
Iteration 2, inertia 15984413.1674
start iteration
done sorting
end inner loop
Iteration 3, inertia 15888779.6377
start iteration
done sorting
end inner loop
Iteration 4, inertia 15818828.7532
start iteration
done sorting
end inner loop
Iteration 5, inertia 15772317.7111
start iteration
done sorting
end inner loop
Iteration 6, inertia 15752404.9416
start iteration
done sorting
end inner loop
Iteration 7, inertia 15751410.6289
start iteration
done sorting
end inner loop
Iteration 8, inertia 15751365.6601
start iteration
done sorting
end inner loop
Iteration 9, inertia 15751365.6601
center shift 0.000000e+00 within tolerance 3.939571e-03
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 16562547.2779
start iteration
done sortin

In [24]:
from collections import Counter
print (len(kmeans.cluster_centers_))


20


/home/harsha/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:1832: RuntimeWarning: invalid value encountered in true_divide
  dm /= _row_norms(XA).reshape(-1, 1)


AttributeError: 'numpy.ndarray' object has no attribute 'index'